This notebook is used to convert the large number of job adverts in JSON format into a single cohesive database that can be exploited, in SQL format.

In [ ]:
import os, json
import pandas as pd
import timeit
import sqlalchemy as alch

In [ ]:
%%time
# Get all json files that are located within the subdirectories
path = './ADS/'
json_files = [os.path.join(root, name)
             for root, dirs, files in os.walk(path)
             for name in files
             if name.endswith(".json")]


In [ ]:
# convert the dataframe to sql and write out to a file
engine = alch.create_engine('sqlite:///jobs.db', echo=False)

In [ ]:
%%time
# build a dataframe with all JSON files, this will take some time
file_number = len(json_files)
print(file_number,"Files total")
chunks = int(file_number/10000)
print(chunks,"chunks of 10k files")



In [ ]:
%%time
i = 0
for c in range(chunks):
    #create a new datafrane for the chunk and process it
    print("Processing chunk",c+1)
    df_list = [None] * 10000
    
    for j in range(0, 10000):
        try:
            data = pd.read_json(json_files[i], typ='series')
            df_list[j] = data
        except:
            df_list[j] = None
        i += 1
  
    dataframe = pd.concat(df_list, axis=1, sort=True)
    dataframe = dataframe.transpose()
    dataframe.to_sql('jobs.db', if_exists='append', con=engine)
    